# Step 3: Multi-Agent Crew Consultation

이 단계에서는 **Multi-Agent System(멀티 에이전트 시스템)**을 구축합니다.
단 하나의 '슈퍼맨' 에이전트 대신, 각자의 전문 분야를 가진 여러 에이전트(Triage, Vet, Advisor)가 협력하여 더 정확하고 풍부한 상담 결과를 만들어내는 과정을 실습합니다.

In [2]:
# 필요한 라이브러리 설치 (최초 1회 실행)
try:
    import crewai
except ImportError:
    print("Installing crewai and dependencies...")
    !pip install crewai pandas openai

import os
import pandas as pd
from crewai import Agent, Task, Crew, Process, LLM

# 환경 변수 설정 (API 키 하드코딩)
os.environ["OPENAI_API_KEY"] = "YOUR_API_KEY_HERE"\nos.environ["OPENAI_MODEL_NAME"] = "gpt-5-mini"

# LLM 설정
my_llm = LLM(model="gpt-5-mini", temperature=1)

## 1. 전문 에이전트 정의 (실습)
세 명의 전문가를 고용합니다.

1.  **Triage Specialist (초진 담당)**: 응급도를 판단하고 분류합니다.
2.  **Veterinary Consultant (진단 담당)**: 증상을 바탕으로 의학적 진단을 내립니다.
3.  **Client Advisor (보호자 상담 담당)**: 어려운 의학 용어를 쉽게 풀어서 보호자에게 전달합니다.

In [3]:
# 1. Triage Agent
triage_agent = Agent(
    role="응급 분류 간호사 (Triage Specialist)",
    goal="증상을 바탕으로 반려동물 상태의 긴급도를 가장 먼저 분류합니다.",
    backstory="""당신은 경험 많은 동물병원 응급실 간호사입니다. 생명이 위급한 징후를 빠르게 파악하고 
    케이스를 위급, 응급, 긴급, 또는 비긴급 카테고리로 신속하게 분류하는 능력을 갖추고 있습니다.""",
    llm=my_llm,
    verbose=True
)

# 2. Vet Consultant Agent
vet_agent = Agent(
    role="진단 수의사 (Veterinary Consultant)",
    goal="증상과 응급도를 파악하여 예상되는 질병과 의학적 설명을 제공합니다.",
    backstory="""당신은 깊은 의학적 지식을 갖춘 전문 수의사입니다. 
    증상을 분석하여 가능한 질병 목록(감별 진단)을 논리적으로 추론해 냅니다.""",
    llm=my_llm,
    verbose=True
)

# 3. Advisor Agent
advisor_agent = Agent(
    role="상담가 (Pet Health Advisor)",
    goal="앞선 분석 결과를 종합하여 보호자를 위해 이해하기 쉽고 따뜻한 조언 리포트를 한국어로 작성합니다.",
    backstory="""당신은 고객 커뮤니케이션 전문가입니다. 복잡한 의학 정보를 
    걱정하는 보호자가 명확하게 이해할 수 있도록 친절하게 설명하고 가정 내 대처법을 덧붙입니다.""",
    llm=my_llm,
    verbose=True
)

## 2. Task 연결 (Workflow 설계)
각 에이전트가 수행할 작업을 정의하고, **이전 작업의 결과(Context)**를 다음 작업이 받을 수 있도록 설정합니다.
CrewAI의 `sequential` 프로세스는 Task 리스트의 순서대로 작업을 실행합니다.

In [4]:
# Task 1: Triage Assessment
triage_task = Task(
    description="""다음 케이스의 긴급도를 분석하세요: {case_input}. 
    Level 1 (위급) 부터 Level 5 (비긴급) 사이로 분류하고 그 이유를 한국어로 짧게 요약하세요.""",
    expected_output="한국어로 작성된 긴급도 분류 수준 및 그 판단 근거.",
    agent=triage_agent
)

# Task 2: Medical Diagnosis (Triage 기반 진단)
diagnosis_task = Task(
    description="""응급 분류 결과와 증상을 바탕으로 예상되는 진단명들을 나열하세요. 
    필요하다면 어떤 검사가 요구되는지도 한국어로 제안해 주세요.""",
    expected_output="한국어로 작성된 예상 질병 목록과 권장되는 의학적 검사 방법.",
    agent=vet_agent,
    context=[triage_task] # Triage 결과 입력 받음
)

# Task 3: Final Advice (최종 조언 작성)
advice_task = Task(
    description="""응급 분류 결과와 수의사의 진단 결과를 꼼꼼히 종합하여 보호자를 위한 최종 답변을 작성하세요. 
    긴급도, 예상 원인, 가정 내 대처법이 반드시 포함되어야 하며, 다정하고 친절한 어조를 사용하여 모든 문장을 한국어로 작성하세요.""",
    expected_output="한국어로 작성된 친절하고 완벽한 수의학 상담 최종 리포트.",
    agent=advisor_agent,
    context=[triage_task, diagnosis_task] # 이전 두 에이전트의 결과를 모두 종합
)

## 3. Crew 실행 (반복)
5개의 케이스에 대해 Multi-Agent 시스템을 실행합니다.

In [ ]:
# 데이터 로드
df_processed = pd.read_csv("./data/processed_sample.csv")

# Crew 생성
multi_agent_crew = Crew(
    agents=[triage_agent, vet_agent, advisor_agent],
    tasks=[triage_task, diagnosis_task, advice_task],
    process=Process.sequential, # 순차 실행
    verbose=True
)

# 반복 실행
for i, row in df_processed.iterrows():
    print(f"\n--- Processing Case #{i+1} ---")
    case_input = row['formatted_prompt']
    
    result = multi_agent_crew.kickoff(inputs={"case_input": case_input})
    
    # 결과 저장
    with open(f"./data/multi_result_{i}.md", "w", encoding="utf-8") as f:
        f.write(result.raw)

print("\nAll cases processed.")


--- Processing Case #1 ---


╭─────────────────────────────────────────── 🚀 Crew Execution Started ───────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name:                                                                                                          │
│  crew                                                                                                           │
│  ID:                                                                                                            │
│  6f2c5457-3780-48a2-b293-6baeb660853f                                                                           │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── 📋 Task Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Started                                                                                                   │
│  Name: 다음 케이스의 긴급도를 분석하세요: Symptom:                                                              │
│  Diarrhea                                                                                                       │
│                                                                                                                 │
│  Pet Information:                                                                                               │
│  Name                                                                                                           │
│  Lucyfur                                                                                                        │
│  Species                                                                                                        │
│  Dog                                                                                                            │
│  Breed                                                                                                          │
│  Portuguese Podengo Pequeno                                                                                     │
│  Age                                                                                                            │
│  3 years                                                                                                        │
│  Gender                                                                                                         │
│  Female                                                                                                         │
│  Status                                                                                                         │
│  Intact                                                                                                         │
│                                                                                                                 │
│  Issue Details:                                                                                                 │
│                                                                                                                 │
│                                                                                                                 │
│  More Details from Guardian:                                                                                    │
│  She has had bad stinky gas for days. Eating grass and puked..                                                  │
│      Level 1 (위급) 부터 Level 5 (비긴급) 사이로 분류하고 그 이유를 한국어로 짧게 요약하세요.                   │
│  ID: 3702b015-8ae6-4243-90a8-61550285a594                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: 응급 분류 간호사 (Triage Specialist)                                                                    │
│                                                                                                                 │
│  Task: 다음 케이스의 긴급도를 분석하세요: Symptom:                                                              │
│  Diarrhea                                                                                                       │
│                                                                                                                 │
│  Pet Information:                                                                                               │
│  Name                                                                                                           │
│  Lucyfur                                                                                                        │
│  Species                                                                                                        │
│  Dog                                                                                                            │
│  Breed                                                                                                          │
│  Portuguese Podengo Pequeno                                                                                     │
│  Age                                                                                                            │
│  3 years                                                                                                        │
│  Gender                                                                                                         │
│  Female                                                                                                         │
│  Status                                                                                                         │
│  Intact                                                                                                         │
│                                                                                                                 │
│  Issue Details:                                                                                                 │
│                                                                                                                 │
│                                                                                                                 │
│  More Details from Guardian:                                                                                    │
│  She has had bad stinky gas for days. Eating grass and puked..                                                  │
│      Level 1 (위급) 부터 Level 5 (비긴급) 사이로 분류하고 그 이유를 한국어로 짧게 요약하세요.                   │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: 응급 분류 간호사 (Triage Specialist)                                                                    │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  레벨 3 (긴급)                                                                                                  │
│                                                                                                                 │
│  이유: 수일간 지속된 악취 나는 가스와 설사, 풀 섭취 후 구토가 있어 소화기 문제(감염, 기생충, 식중독, 이물       │
│  등)로 진행되었을 가능성이 있고 탈수·전해질 이상으로 악화될 수 있으므로 24시간 이내 수의사 진찰이 필요합니다.   │
│  혈변·반복구토·무기력·식욕부진·탈수 징후가 있으면 즉시 응급처치(레벨 1–2)가 필요합니다.                         │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: 응급 분류 간호사 (Triage Specialist)                                                                    │
│                                                                                                                 │
│  Task: 다음 케이스의 긴급도를 분석하세요: Symptom:                                                              │
│  Diarrhea                                                                                                       │
│                                                                                                                 │
│  Pet Information:                                                                                               │
│  Name                                                                                                           │
│  Ivy                                                                                                            │
│  Species                                                                                                        │
│  Dog                                                                                                            │
│  Breed                                                                                                          │
│  Dachshund                                                                                                      │
│  Age                                                                                                            │
│  13 years                                                                                                       │
│  Gender                                                                                                         │
│  Female                                                                                                         │
│  Status                                                                                                         │
│  Neutered or spayed                                                                                             │
│                                                                                                                 │
│  Issue Details:                                                                                                 │
│                                                                                                                 │
│                                                                                                                 │
│  More Details from Guardian:                                                                                    │
│  She only had bad orangish diarhea one day. The next day she would not eat anything and she just layed around   │
│  yesterday i noticed her bresthing was shallow still not eating and laying around. Today shes wheezing and      │
│  wont walk and she looks a little swollen.                                                                      │
│      Level 1 (위급) 부터 Level 5 (비긴급) 사이로 분류하고 그 이유를 한국어로 짧게 요약하세요.                   │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: 응급 분류 간호사 (Triage Specialist)                                                                    │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  레벨 1 (위급)                                                                                                  │
│                                                                                                                 │
│  이유: 고령(13세) 개에서 설사 후 식욕부진·무기력·보행불가, 얕은 호흡과 쌕쌕거림(호흡곤란 증상), 몸의            │
│  붓기(부종/안구/얼굴 등)가 동반되어 있어 패혈증, 급성 심부전/폐부종, 아나필락시스 등 생명 위협적 상태 가능성이  │
│  큽니다. 즉시 응급수의사 진찰과 산소치료·안정·혈액검사·수액 등 응급처치가 필요하므로 즉시 응급병원으로 가세요.  │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────── 📋 Task Completion ───────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name:                                                                                                          │
│  다음 케이스의 긴급도를 분석하세요: Symptom:                                                                    │
│  Diarrhea                                                                                                       │
│                                                                                                                 │
│  Pet Information:                                                                                               │
│  Name                                                                                                           │
│  Ivy                                                                                                            │
│  Species                                                                                                        │
│  Dog                                                                                                            │
│  Breed                                                                                                          │
│  Dachshund                                                                                                      │
│  Age                                                                                                            │
│  13 years                                                                                                       │
│  Gender                                                                                                         │
│  Female                                                                                                         │
│  Status                                                                                                         │
│  Neutered or spayed                                                                                             │
│                                                                                                                 │
│  Issue Details:                                                                                                 │
│                                                                                                                 │
│                                                                                                                 │
│  More Details from Guardian:                                                                                    │
│  She only had bad orangish diarhea one day. The next day she would not eat anything and she just layed around   │
│  yesterday i noticed her bresthing was shallow still not eating and laying around. Today shes wheezing and      │
│  wont walk and she looks a little swollen.                                                                      │
│      Level 1 (위급) 부터 Level 5 (비긴급) 사이로 분류하고 그 이유를 한국어로 짧게 요약하세요.                   │
│  Agent:                                                                                                         │
│  응급 분류 간호사 (Triage Specialist)                                                                           │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── 📋 Task Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Started                                                                                                   │
│  Name: 응급 분류 결과와 증상을 바탕으로 예상되는 진단명들을 나열하세요.                                         │
│      필요하다면 어떤 검사가 요구되는지도 한국어로 제안해 주세요.                                                │
│  ID: 3ef1aaa9-5b3e-496c-b8df-b1f706269078                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: 진단 수의사 (Veterinary Consultant)                                                                     │
│                                                                                                                 │
│  Task: 응급 분류 결과와 증상을 바탕으로 예상되는 진단명들을 나열하세요.                                         │
│      필요하다면 어떤 검사가 요구되는지도 한국어로 제안해 주세요.                                                │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯